In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
import numpy as np
import tensorflow as tf

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

dataset_path = os.listdir('/content/drive/MyDrive/Colab Notebooks/PlantVillage')

print (dataset_path)  #what kinds of classes are in this dataset

print("Types of classes labels found: ", len(dataset_path))

['Potato___Early_blight', 'Potato___healthy', 'Potato___Late_blight']
Types of classes labels found:  3


In [13]:
class_labels = []

for item in dataset_path:
 # Get all the file names
 all_classes = os.listdir('/content/drive/MyDrive/Colab Notebooks/PlantVillage' + '/' +item)
 #print(all_classes)

 # Add them to the list
 for room in all_classes:
    class_labels.append((item, str('dataset_path' + '/' +item) + '/' + room))
    #print(class_labels[:5])

In [14]:

# Build a dataframe
df = pd.DataFrame(data=class_labels, columns=['Labels', 'image'])
print(df.head())
print(df.tail())

                  Labels                                              image
0  Potato___Early_blight  dataset_path/Potato___Early_blight/001187a0-57...
1  Potato___Early_blight  dataset_path/Potato___Early_blight/002a55fb-7a...
2  Potato___Early_blight  dataset_path/Potato___Early_blight/009c8c31-f2...
3  Potato___Early_blight  dataset_path/Potato___Early_blight/00d8f10f-50...
4  Potato___Early_blight  dataset_path/Potato___Early_blight/0182e991-97...
                    Labels                                              image
2147  Potato___Late_blight  dataset_path/Potato___Late_blight/fe9d57f1-8e0...
2148  Potato___Late_blight  dataset_path/Potato___Late_blight/feefc118-443...
2149  Potato___Late_blight  dataset_path/Potato___Late_blight/ff03a125-445...
2150  Potato___Late_blight  dataset_path/Potato___Late_blight/ff7160f9-f8d...
2151  Potato___Late_blight  dataset_path/Potato___Late_blight/fff55a7e-2f8...


In [15]:
# Let's check how many samples for each category are present
print("Total number of images in the dataset: ", len(df))

label_count = df['Labels'].value_counts()
print(label_count)

Total number of images in the dataset:  2152
Potato___Early_blight    1000
Potato___Late_blight     1000
Potato___healthy          152
Name: Labels, dtype: int64


In [16]:
import cv2
path = '/content/drive/MyDrive/Colab Notebooks/PlantVillage/'
dataset_path = os.listdir('/content/drive/MyDrive/Colab Notebooks/PlantVillage')

im_size = 224

images = []
labels = []

for i in dataset_path:
    data_path = path + str(i)
    filenames = [i for i in os.listdir(data_path) ]

    for f in filenames:
        img = cv2.imread(data_path + '/' + f)
        img = cv2.resize(img, (im_size, im_size))
        images.append(img)
        labels.append(i)

In [17]:

#This model takes input images of shape (224, 224, 3), and the input data should range [0, 255].

images = np.array(images)

images = images.astype('float32') / 255.0
images.shape

(2152, 224, 224, 3)

In [18]:
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
y=df['Labels'].values
# print(y)
y_u = np.unique(y)
print(y_u)

y_labelencoder = LabelEncoder ()
y = y_labelencoder.fit_transform (y)
# print (y)
y_u = np.unique(y)
print(y_u)

['Potato___Early_blight' 'Potato___Late_blight' 'Potato___healthy']
[0 1 2]


In [19]:

y=y.reshape(-1,1)

from sklearn.compose import ColumnTransformer
ct = ColumnTransformer([('my_ohe', OneHotEncoder(), [0])], remainder='passthrough')
Y = ct.fit_transform(y) #.toarray()
print(Y[:5])
print(Y[35:])

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [20]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


images, Y = shuffle(images, Y, random_state=1)


train_x, test_x, train_y, test_y = train_test_split(images, Y, test_size=0.05, random_state=415)

#inpect the shape of the training and testing.
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)



(2044, 224, 224, 3)
(2044, 3)
(108, 224, 224, 3)
(108, 3)


In [23]:
from keras.preprocessing.image import ImageDataGenerator

train_generatorENB0 = ImageDataGenerator(preprocessing_function=lambda x: (x / 127.5) - 1.0,
                                         rotation_range=40,
                                         width_shift_range=0.25,
                                         height_shift_range=0.25,
                                         shear_range=0.25,
                                         zoom_range=0.25,
                                         horizontal_flip=True,
                                         fill_mode='nearest',)


test_generatorENB0 = ImageDataGenerator(preprocessing_function=lambda x: (x / 127.5) - 1.0)

train_generatorENB0.fit(train_x)
test_generatorENB0.fit(test_x)

In [24]:
import tensorflow as tf
effNet = tf.keras.applications.EfficientNetB0(
                            include_top=False,
                            weights='imagenet',
)

# effNet.trainable=False

resNet=tf.keras.applications.resnet50.ResNet50(include_top=False,
                             weights='imagenet',
                             )
# resNet.trainable=False


input_shape = (224,224, 3)

inputs = tf.keras.layers.Input(shape=input_shape, name="input_layer")

x = effNet(inputs, training=True)
y = resNet(inputs, training=True)

x = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer1")(x)
y = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer2")(y)

x = tf.keras.layers.Concatenate(axis=1)([x, y])

x = tf.keras.layers.Dense(1028, activation="relu", name="hidden")(x)
outputs = tf.keras.layers.Dense(3, activation="softmax", name="output_layer")(x)

hybrid_model = tf.keras.Model(inputs, outputs)

hybrid_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(hybrid_model.summary())


94765736/94765736 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_layer (InputLayer)    [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 efficientnetb0 (Functional  (None, None, None, 1280)     4049571   ['input_layer[0][0]']         
 )                                                                                                
                                                                                                  
 resnet50 (Functional)       (None, None, None, 2048)     2358771   ['input_layer[0][0]']         
                                                          2                                       
                             

In [26]:
history = hybrid_model.fit(train_x, train_y, epochs=10, verbose=2)

Epoch 1/10
64/64 - 42s - loss: 0.2367 - accuracy: 0.9560 - 42s/epoch - 654ms/step
Epoch 2/10
64/64 - 40s - loss: 0.0384 - accuracy: 0.9883 - 40s/epoch - 630ms/step
Epoch 3/10
64/64 - 40s - loss: 0.0383 - accuracy: 0.9917 - 40s/epoch - 629ms/step
Epoch 4/10
64/64 - 40s - loss: 0.0171 - accuracy: 0.9941 - 40s/epoch - 630ms/step
Epoch 5/10
64/64 - 40s - loss: 0.0017 - accuracy: 0.9995 - 40s/epoch - 630ms/step
Epoch 6/10
64/64 - 40s - loss: 0.0400 - accuracy: 0.9936 - 40s/epoch - 629ms/step
Epoch 7/10
64/64 - 40s - loss: 0.0372 - accuracy: 0.9907 - 40s/epoch - 630ms/step
Epoch 8/10
64/64 - 40s - loss: 0.0040 - accuracy: 0.9980 - 40s/epoch - 630ms/step
Epoch 9/10
64/64 - 40s - loss: 0.0130 - accuracy: 0.9961 - 40s/epoch - 629ms/step
Epoch 10/10
64/64 - 40s - loss: 0.0261 - accuracy: 0.9932 - 40s/epoch - 628ms/step


In [27]:
preds = hybrid_model.evaluate(test_x, test_y)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

4/4 [==============================] - 4s 409ms/step - loss: 0.0379 - accuracy: 0.9907
Loss = 0.03791128098964691
Test Accuracy = 0.9907407164573669
